In [1]:
pip install praw vaderSentiment tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 7.4 MB/s eta 0:00:00


In [33]:
import praw
from tqdm import tqdm
import csv
import pandas as pd

import warnings

# Suppress specific warning type (e.g., DeprecationWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [3]:
# 2. Initialize Reddit API client using PRAW
reddit = praw.Reddit(
    client_id='pPsx95TX-i0nWFf0_jXygg',
    client_secret='OSwboGqfZY_-MkgxAjGgGIyjxX_DZA',
    user_agent='testscript by u/fakebot3'
)

In [5]:
df=pd.read_csv('df.csv')

In [ ]:
import numpy as np
from tqdm import tqdm

def get_reddit_posts(tagword, limit=10):
    """
    Searches Reddit for posts related to the given tagword.
    Returns a list of dictionaries with post details.
    """
    posts = []
    for submission in reddit.subreddit('movies').search(tagword, limit=limit, sort='relevance'):
        posts.append({
            'title': submission.title,
            'selftext': submission.selftext,
            'subreddit': submission.subreddit.display_name,
            'url': submission.url
        })

    # If no posts are found, return NaN entries
    if not posts:
        posts.append({
            'title': np.nan,
            'selftext': np.nan,
            'subreddit': np.nan,
            'url': np.nan
        })
    return posts

results = []
for movie_id, movie_title in tqdm(df[['movie_id', 'movie_title']].drop_duplicates().values[999:], desc="Fetching Reddit Posts"): #can only scrape 999 at one time hence , we run this function twice with[:999] and [999:]
    #tagword = movie_title.split('(')[0].strip()  # generate tagword from movie title
    tagword = movie_title

    # Get related posts from Reddit
    posts = get_reddit_posts(tagword, limit=5)

    # If no posts were found, make sure to append a NaN post
    if not posts:
        posts = [{
            'movie_id': movie_id,
            'movie_title': movie_title,
            'title': np.nan,
            'selftext': np.nan,
            'subreddit': np.nan,
            'url': np.nan
        }]

    for post in posts:
        post['movie_id'] = movie_id
        post['movie_title'] = movie_title
        results.append(post)



In [65]:
# Check if the number of unique movie_ids remains the same
unique_movie_ids = len(set([result['movie_id'] for result in results]))
print(f"Unique movie_ids: {unique_movie_ids}")

Unique movie_ids: 683


In [41]:
df_1= pd.DataFrame(results)
df_1.to_csv("reddit_posts_2.csv", index=False)

In [52]:
df_1=pd.read_csv('reddit_posts_1.csv')
df_2=pd.read_csv('reddit_posts_2.csv')
df_sentiments=pd.concat([df_1,df_2])

In [53]:
df_sentiments['movie_id'].nunique()

1682

In [54]:
df_sentiments.isna().sum()

,0
title,151
selftext,683
subreddit,151
url,151
movie_id,0
movie_title,0


In [55]:
df_sentiments['review'] = df_sentiments['title'].fillna('') + df_sentiments['selftext'].fillna('')

In [57]:
col_drop=['title','selftext','subreddit','url']
df_sentiments.drop(col_drop,axis=1,inplace=True)

In [64]:
df_sentiments.sample(5)

,movie_id,movie_title,review
3632,804,Jimmy Hollywood (1994),Full Daily Turner Classic Movie (U.S.) Schedul...
1993,1549,Dream Man (1995),List of YouTube Free Movies w/ Rotten Tomatoes...
1693,1456,Beat the Devil (1954),Turner Classic Movies (U.S.) Full Daily Schedu...
3643,806,Menace II Society (1993),Best year for film in the last 50 years?I’ve g...
3141,690,Seven Years in Tibet (1997),123 lesser-known movies from 1976-2016 that ar...


In [62]:
df_sentiments.to_csv('reviews.csv',index=False)